# Generate Deep Review stats for templated use in the Meta Review

In [1]:
import collections
import datetime
import json
import pathlib

import pandas
import requests

In [2]:
stats = collections.OrderedDict()
stats['creation_time_utc'] = datetime.datetime.utcnow().isoformat()

## Deep Review git repository stats

In [3]:
commit_df = pandas.read_table('commits.tsv')
commit_df.tail(2)

,commit,author_name,author_email,committer_name,authored_datetime,committed_datetime,summary,count,merge,parents,words_added,words_deleted,characters_added,characters_deleted
436,2e721e0c9ee8314688581e0e4dd465222d867b01,Yifan Peng,yifan.peng@nih.gov,Anthony Gitter,2017-10-04 07:20:57-04:00,2017-10-04 06:20:57-05:00,Add ChestX-ray data link (#668) closes #666,437,0,6d95f407b414ee84c899150f7f9a4482e3c98fb5,13,2,167,13
437,d4c316393779a9248d924abae83bc672aafa39df,Yifan Peng,yifan.peng@nih.gov,Anthony Gitter,2017-10-04 14:34:28-04:00,2017-10-04 13:34:28-05:00,ChestX-ray dataset reference formatting (#669)...,438,0,2e721e0c9ee8314688581e0e4dd465222d867b01,0,0,0,0


In [4]:
# State of the git repo (deep-review submodule)
stats['commit'] = commit_df.commit.iloc[-1]

In [5]:
stats['total_commits'] = len(commit_df)

In [6]:
# Number of non-merge commits that modified the manuscript markdown source
writing_commit_df = commit_df.query("(characters_added > 0 or characters_deleted > 0) and merge == 0")
stats['manuscript_commits'] = len(writing_commit_df)

### Number of formal Deep Review authors

In [7]:
author_df = pandas.read_table('deep-review/authors.tsv')
stats['deep_review_authors'] = len(author_df)

## GitHub repo stats

In [8]:
# https://developer.github.com/v3/repos/#get
response = requests.get('https://api.github.com/repos/greenelab/deep-review')
result = response.json()
stats['github_stars'] = result['stargazers_count']
stats['github_forks'] = result['forks_count']

### Number of pull requests

In [9]:
def github_issue_search(query):
    """
    Search issues and pull requests on GitHub.

    https://developer.github.com/v3/search/#search-issues
    https://help.github.com/articles/searching-issues-and-pull-requests/
    """
    url = 'https://api.github.com/search/issues'
    params = {
        'q': query,
        'sort': 'created',
        'order': 'asc',
    }
    response = requests.get(url, params)
    print(response.url)
    assert response.status_code == 200
    result = response.json()
    assert not result['incomplete_results']
    return result

In [10]:
# Merged PRs
result = github_issue_search('repo:greenelab/deep-review type:pr is:merged')
stats['merged_pull_requests'] = result['total_count']

https://api.github.com/search/issues?q=repo%3Agreenelab%2Fdeep-review+type%3Apr+is%3Amerged&sort=created&order=asc


In [11]:
# Closed PRs that were not merged
result = github_issue_search('repo:greenelab/deep-review type:pr is:unmerged state:closed')
stats['declined_pull_requests'] = result['total_count']

https://api.github.com/search/issues?q=repo%3Agreenelab%2Fdeep-review+type%3Apr+is%3Aunmerged+state%3Aclosed&sort=created&order=asc


In [12]:
# Open PRs
result = github_issue_search('repo:greenelab/deep-review type:pr state:open')
stats['open_pull_requests'] = result['total_count']

https://api.github.com/search/issues?q=repo%3Agreenelab%2Fdeep-review+type%3Apr+state%3Aopen&sort=created&order=asc


In [13]:
# Open Issues
result = github_issue_search('repo:greenelab/deep-review type:issue state:open')
stats['open_issues'] = result['total_count']

https://api.github.com/search/issues?q=repo%3Agreenelab%2Fdeep-review+type%3Aissue+state%3Aopen&sort=created&order=asc


In [14]:
# Closed Issues
result = github_issue_search('repo:greenelab/deep-review type:issue state:closed')
stats['closed_issues'] = result['total_count']

https://api.github.com/search/issues?q=repo%3Agreenelab%2Fdeep-review+type%3Aissue+state%3Aclosed&sort=created&order=asc


## Write stats

In [15]:
stats_str = json.dumps(stats, indent=2)
print(stats_str)

{
  "creation_time_utc": "2017-10-30T20:27:57.162833",
  "commit": "d4c316393779a9248d924abae83bc672aafa39df",
  "total_commits": 438,
  "manuscript_commits": 253,
  "deep_review_authors": 27,
  "github_stars": 344,
  "github_forks": 113,
  "merged_pull_requests": 225,
  "declined_pull_requests": 19,
  "open_pull_requests": 10,
  "open_issues": 333,
  "closed_issues": 88
}


In [16]:
path = pathlib.Path('deep-review-stats.json')
path.write_text(stats_str)

375